# 04 Performance Profiling

**Objective:** Ensure system performs well with production-scale data.

**Areas to Profile:**
- Query performance (database operations)
- Code performance (Python/pandas operations)

**Why This Matters:** System must handle 80K+ reviews efficiently for production use.

In [1]:
import sys
from pathlib import Path
project_root = Path.cwd() if (Path.cwd() / "src").exists() else Path.cwd().parent
sys.path.insert(0, str(project_root))

import time
import io
import sqlite3
from contextlib import redirect_stdout
import pandas as pd
import cProfile
import pstats

from src.utils import get_db_path
from src.data_processing import REVIEWS_INDEXES
from src.benchmarking import get_reviews_df, create_comparable_groups, extract_hotel_features

db_path = get_db_path(sample=False)
NUM_RUNS = 5

## Query Performance: Quantified Improvements (Baseline vs With Indexes)

We measure **before** and **after** adding indexes to show quantified improvement (assignment: "Performance profiling with quantified improvements").

**Steps:** Drop indexes → run queries (baseline) → create indexes → run same queries (with indexes) → report improvement %.  
**Note:** This cell sequence modifies the DB in place (drops then recreates indexes). End state: indexes present.

**Test queries:** Count, Group by hotel, Filter by rating, Filter by offering_id (dashboard-style lookup), Complex aggregation.  
Each query runs 5 times; we report average baseline ms, average with-index ms, and improvement %.

In [2]:
# Get a sample offering_id for the "Filter by offering_id" query (dashboard-style lookup)
conn = sqlite3.connect(str(db_path))
sample_offering_id = conn.execute("SELECT offering_id FROM reviews LIMIT 1").fetchone()[0]
conn.close()

test_queries = [
    ("Count all reviews", "SELECT COUNT(*) FROM reviews"),
    ("Avg rating by hotel", "SELECT offering_id, AVG(rating_overall) FROM reviews GROUP BY offering_id"),
    ("Filter by rating >= 4", "SELECT * FROM reviews WHERE rating_overall >= 4 LIMIT 1000"),
    ("Filter by offering_id", f"SELECT * FROM reviews WHERE offering_id = {sample_offering_id} LIMIT 1000"),
    ("Complex aggregation", """
        SELECT offering_id,
               COUNT(*) as n,
               AVG(rating_overall) as avg_rating,
               AVG(rating_cleanliness) as avg_clean
        FROM reviews
        WHERE rating_overall >= 3.5
        GROUP BY offering_id
        HAVING COUNT(*) >= 10
        ORDER BY avg_rating DESC
        LIMIT 100
    """),
]

# Index names to drop (must match REVIEWS_INDEXES in data_processing.py)
DROP_INDEXES = [
    "DROP INDEX IF EXISTS idx_reviews_offering;",
    "DROP INDEX IF EXISTS idx_reviews_author;",
    "DROP INDEX IF EXISTS idx_reviews_rating_overall;",
    "DROP INDEX IF EXISTS idx_reviews_offering_rating_clean;",
]

### Step 1: Baseline (no indexes)

Drop indexes, then run each query 5 times and record average/min/max.

In [3]:
def run_query_timing(conn, test_queries, num_runs):
    """Run each query num_runs times; return list of {query, avg_time, min_time, max_time}."""
    results = []
    for name, query in test_queries:
        times = []
        for _ in range(num_runs):
            start = time.time()
            conn.execute(query).fetchall()
            times.append(time.time() - start)
        avg_time = sum(times) / len(times)
        results.append({"query": name, "avg_time": avg_time, "min_time": min(times), "max_time": max(times)})
    return results

# Drop indexes (baseline = no indexes)
conn = sqlite3.connect(str(db_path))
for stmt in DROP_INDEXES:
    conn.execute(stmt)
conn.commit()

results_baseline = run_query_timing(conn, test_queries, NUM_RUNS)
conn.close()
print("Baseline (no indexes):")
for r in results_baseline:
    print(f"  {r['query']}: avg={r['avg_time']*1000:.2f}ms")

Baseline (no indexes):
  Count all reviews: avg=0.40ms
  Avg rating by hotel: avg=341.37ms
  Filter by rating >= 4: avg=13.31ms
  Filter by offering_id: avg=210.45ms
  Complex aggregation: avg=294.57ms


### Step 2: With indexes

Recreate indexes (same as data_processing.REVIEWS_INDEXES), then run same queries 5 times.

In [4]:
# Create indexes (same as data_processing.REVIEWS_INDEXES)
conn = sqlite3.connect(str(db_path))
for stmt in REVIEWS_INDEXES:
    conn.execute(stmt)
conn.commit()

results_with_index = run_query_timing(conn, test_queries, NUM_RUNS)
conn.close()
print("With indexes:")
for r in results_with_index:
    print(f"  {r['query']}: avg={r['avg_time']*1000:.2f}ms")

With indexes:
  Count all reviews: avg=0.40ms
  Avg rating by hotel: avg=10.71ms
  Filter by rating >= 4: avg=15.51ms
  Filter by offering_id: avg=0.40ms
  Complex aggregation: avg=11.20ms


In [5]:
# Summary: quantified improvement (baseline vs with indexes)
summary_rows = []
for b, w in zip(results_baseline, results_with_index):
    baseline_ms = b["avg_time"] * 1000
    with_index_ms = w["avg_time"] * 1000
    improvement = (1 - with_index_ms / baseline_ms) * 100 if baseline_ms > 0 else 0
    summary_rows.append({
        "Query": b["query"],
        "Baseline (ms)": round(baseline_ms, 2),
        "With indexes (ms)": round(with_index_ms, 2),
        "Improvement (%)": round(improvement, 1),
    })
summary_df = pd.DataFrame(summary_rows)
print(summary_df.to_string(index=False))

# Write profiling output (baseline + with-index + summary) to file
buf = io.StringIO()
buf.write("QUERY PERFORMANCE: BASELINE (no indexes) vs WITH INDEXES\n")
buf.write("=" * 80 + "\n\n")
buf.write("Baseline (no indexes):\n")
for r in results_baseline:
    buf.write(f"  {r['query']}: avg={r['avg_time']*1000:.2f}ms, min={r['min_time']*1000:.2f}ms, max={r['max_time']*1000:.2f}ms\n")
buf.write("\nWith indexes:\n")
for r in results_with_index:
    buf.write(f"  {r['query']}: avg={r['avg_time']*1000:.2f}ms, min={r['min_time']*1000:.2f}ms, max={r['max_time']*1000:.2f}ms\n")
buf.write("\nQUANTIFIED IMPROVEMENT:\n")
buf.write(summary_df.to_string(index=False) + "\n")
query_profiling_txt = buf.getvalue()
output_path = project_root / "profiling" / "query_results.txt"
output_path.parent.mkdir(parents=True, exist_ok=True)
output_path.write_text(query_profiling_txt, encoding="utf-8")
print(f"\nWrote {output_path}")

                Query  Baseline (ms)  With indexes (ms)  Improvement (%)
    Count all reviews           0.40               0.40             -0.1
  Avg rating by hotel         341.37              10.71             96.9
Filter by rating >= 4          13.31              15.51            -16.5
Filter by offering_id         210.45               0.40             99.8
  Complex aggregation         294.57              11.20             96.2

Wrote d:\dev\study\IS5126-G4-hotel-analytics\profiling\query_results.txt


### Step 3: EXPLAIN QUERY PLAN (with indexes)

Execution plans after indexes are in place (justifies index usage).

In [6]:
print("=" * 70)
print("QUERY EXECUTION PLANS (with indexes)")
print("=" * 70)
conn = sqlite3.connect(str(db_path))
for name, query in test_queries:
    print(f"\n{name}:")
    print(f"  Query: {query[:80]}{'...' if len(query) > 80 else ''}")
    print("  Plan:")
    for row in conn.execute(f"EXPLAIN QUERY PLAN {query}").fetchall():
        print(f"    {row}")
conn.close()

QUERY EXECUTION PLANS (with indexes)

Count all reviews:
  Query: SELECT COUNT(*) FROM reviews
  Plan:
    (4, 0, 0, 'SCAN reviews USING COVERING INDEX idx_reviews_rating_overall')

Avg rating by hotel:
  Query: SELECT offering_id, AVG(rating_overall) FROM reviews GROUP BY offering_id
  Plan:
    (6, 0, 0, 'SCAN reviews USING COVERING INDEX idx_reviews_offering_rating_clean')

Filter by rating >= 4:
  Query: SELECT * FROM reviews WHERE rating_overall >= 4 LIMIT 1000
  Plan:
    (4, 0, 0, 'SEARCH reviews USING INDEX idx_reviews_rating_overall (rating_overall>?)')

Filter by offering_id:
  Query: SELECT * FROM reviews WHERE offering_id = 72572 LIMIT 1000
  Plan:
    (4, 0, 0, 'SEARCH reviews USING INDEX idx_reviews_offering_rating_clean (offering_id=?)')

Complex aggregation:
  Query: 
        SELECT offering_id,
               COUNT(*) as n,
               AVG(ra...
  Plan:
    (8, 0, 0, 'SCAN reviews USING COVERING INDEX idx_reviews_offering_rating_clean')
    (56, 0, 0, 'USE TEMP B-TR

## Code Performance Analysis

Profiling Python operations to identify bottlenecks in our benchmarking code.

**What we're profiling:** The complete benchmarking workflow (feature extraction + clustering).

In [7]:
# Use runctx() so profiling is self-contained (no enable/disable state; avoids "another profiler active")
def _run_benchmark_workflow():
    df = get_reviews_df(sample=False)
    features = extract_hotel_features(df)
    features_clustered, sil_score, profiles = create_comparable_groups(features, n_clusters=6)
    return None

prof = cProfile.Profile()
prof.runctx("_run_benchmark_workflow()", globals(), locals())

# Generate report
s = io.StringIO()
ps = pstats.Stats(prof, stream=s).strip_dirs().sort_stats("cumulative")
ps.print_stats(30)

code_profiling_txt = s.getvalue()

# Write to file
output_path = project_root / "profiling" / "code_profiling.txt"
output_path.write_text(code_profiling_txt, encoding="utf-8")

# Show in notebook
print(code_profiling_txt)
print(f"\n Wrote {output_path}")

Extracting hotel-level features...
Analyzing review text for hotel characteristics...
Creating 6 comparable groups...
Data shape: (3374, 7)
Unique hotels: 3374

Testing different cluster counts:
  K=3: silhouette=0.297, min_size=459, max_size=1487
  K=4: silhouette=0.306, min_size=258, max_size=1341
  K=5: silhouette=0.291, min_size=105, max_size=1366
  K=6: silhouette=0.303, min_size=69, max_size=892
  K=7: silhouette=0.315, min_size=45, max_size=826
  K=8: silhouette=0.333, min_size=75, max_size=926
  K=9: silhouette=0.333, min_size=15, max_size=776
  K=10: silhouette=0.310, min_size=15, max_size=740
  K=11: silhouette=0.314, min_size=14, max_size=741
  K=12: silhouette=0.281, min_size=14, max_size=588

 Selected K=9 with silhouette=0.333
         3856159 function calls (3779609 primitive calls) in 25.891 seconds

   Ordered by: cumulative time
   List reduced from 2108 to 30 due to restriction <30>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
     30/1   